<a href="https://colab.research.google.com/github/yohannes-taye/AAU_CS_IntroductionToOpenGl/blob/main/transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pydub
!apt-get install ffmpeg
!pip install yt-dlp webvtt-py
!pip install googletrans==4.0.0-rc1
!pip install requests
!pip install pypinyin
!pip install jieba


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


### List all available subtitles from the video. If youre looking to generate chinese audio cards then look for `zh-Hans`

In [ ]:
# Download subtitles (vtt format)
#!yt-dlp --write-auto-sub --sub-lang en --sub-format vtt --skip-download https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s
!yt-dlp --list-subs https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s
#should appear as zh-Hans

### Download transcript. Add language using the flag `--sub-lang zh-Hans`

In [5]:
!yt-dlp --write-sub --sub-lang zh-Hans --sub-format vtt --skip-download https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s


[youtube] Extracting URL: https://www.youtube.com/watch?v=g1LsFSO1mbY
[youtube] g1LsFSO1mbY: Downloading webpage
[youtube] g1LsFSO1mbY: Downloading tv client config
[youtube] g1LsFSO1mbY: Downloading player 9a279502-main
[youtube] g1LsFSO1mbY: Downloading tv player API JSON
[youtube] g1LsFSO1mbY: Downloading ios player API JSON
[youtube] g1LsFSO1mbY: Downloading m3u8 information
[info] g1LsFSO1mbY: Downloading subtitles: zh-Hans
[info] g1LsFSO1mbY: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: 史上最值钱的那些公司 了解一下~ [g1LsFSO1mbY].zh-Hans.vtt
[download] Destination: 史上最值钱的那些公司 了解一下~ [g1LsFSO1mbY].zh-Hans.vtt
[download] 100% of   34.04KiB in 00:00:00 at 205.89KiB/s


### Download EditThisCOokie extention for chrome and export cookie for youtube video. use netscape format.

### Then the script below will download the audio file

Download link:
https://chromewebstore.google.com/detail/editthiscookie-v3/ojfebgpkimhlhcblbalbfjblapadhbol

In [8]:
import webvtt
import json

# Function to decode unicode escape sequences properly
def decode_unicode_escape(text):
    # Ensuring proper decoding if needed
    return text.encode('utf-8').decode('utf-8')

def vtt_to_json(vtt_file, json_out):
    transcript = []

    # Read the .vtt file using webvtt
    for caption in webvtt.read(vtt_file):
        start = convert_timestamp(caption.start)
        end = convert_timestamp(caption.end)

        # Directly assign text, as it's already decoded correctly by webvtt
        decoded_text = caption.text.strip().replace('\n', ' ')

        # Append the caption data to the transcript
        transcript.append({
            "start": start,
            "end": end,
            "text": decoded_text
        })

    # Write the decoded transcript to a JSON file with proper encoding
    with open(json_out, 'w', encoding='utf-8') as f:
        json.dump(transcript, f, indent=2, ensure_ascii=False)

def convert_timestamp(timestamp):
    h, m, s = timestamp.split(":")
    s, ms = s.split(".")
    return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000.0

# Example usage
vtt_file = "/content/史上最值钱的那些公司 了解一下~ [g1LsFSO1mbY].zh-Hans.vtt"  # Replace with actual .vtt file path
json_out = "transcript.json"
vtt_to_json(vtt_file, json_out)


In [6]:
!yt-dlp --cookies /content/cookie.txt -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 --output "%(title)s.%(ext)s" https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s


[youtube] Extracting URL: https://www.youtube.com/watch?v=g1LsFSO1mbY
[youtube] g1LsFSO1mbY: Downloading webpage
[youtube] g1LsFSO1mbY: Downloading tv client config
[youtube] g1LsFSO1mbY: Downloading player 9a279502-main
[youtube] g1LsFSO1mbY: Downloading tv player API JSON
[info] g1LsFSO1mbY: Downloading 1 format(s): 251
[download] Destination: 史上最值钱的那些公司 了解一下~.webm
[download] 100% of   17.85MiB in 00:00:00 at 27.82MiB/s
[ExtractAudio] Destination: 史上最值钱的那些公司 了解一下~.mp3
Deleting original file 史上最值钱的那些公司 了解一下~.webm (pass -k to keep)


### Split audio using provided transcript.

In [24]:
from pydub import AudioSegment
import os
import json

# Load the transcript JSON file
with open('transcript.json', 'r', encoding='utf-8') as f:
    transcript = json.load(f)

# Load the audio file (replace with the actual file name)
audio_file = '/content/史上最值钱的那些公司 了解一下~.mp3'  # Replace with the path to your audio file
audio = AudioSegment.from_file(audio_file)

# Create an output directory to save the audio chunks
output_dir = "/content/split_audio"
os.makedirs(output_dir, exist_ok=True)

# Initialize a variable to hold the combined audio chunks
combined_audio = None
combined_text = ""
chunk_counter = 0

# Loop through the transcript and split the audio in pairs
for i in range(0, len(transcript) - 1, 2):  # Step by 2 to process pairs
    # Get the start and end times for the current and next sentence
    start_ms_1 = int(transcript[i]["start"] * 1000)  # Convert to milliseconds
    end_ms_1 = int(transcript[i]["end"] * 1000)  # Convert to milliseconds
    text_1 = transcript[i]["text"]

    start_ms_2 = int(transcript[i+1]["start"] * 1000)  # Convert to milliseconds
    end_ms_2 = int(transcript[i+1]["end"] * 1000)  # Convert to milliseconds
    text_2 = transcript[i+1]["text"]

    # Extract the audio chunks for both entries
    chunk_1 = audio[start_ms_1:end_ms_1]
    chunk_2 = audio[start_ms_2:end_ms_2]

    # Combine the two chunks
    combined_audio = chunk_1 + chunk_2

    # Combine the text from both entries
    combined_text = text_1 + text_2

    # Save the combined audio chunk as a new file
    combined_filename = f"combined_chunk_{chunk_counter:04d}.mp3"  # File name format
    combined_path = os.path.join(output_dir, combined_filename)
    combined_audio.export(combined_path, format="mp3")

    # Save the corresponding combined text as a .txt file
    with open(os.path.join(output_dir, f"combined_chunk_{chunk_counter:04d}.txt"), 'w', encoding='utf-8') as txt_file:
        txt_file.write(combined_text)

    # Print confirmation for the saved file
    print(f"Saved: {combined_filename}")

    # Increment the counter for the next combined chunk
    chunk_counter += 1



Saved: combined_chunk_0000.mp3
Saved: combined_chunk_0001.mp3
Saved: combined_chunk_0002.mp3
Saved: combined_chunk_0003.mp3
Saved: combined_chunk_0004.mp3
Saved: combined_chunk_0005.mp3
Saved: combined_chunk_0006.mp3
Saved: combined_chunk_0007.mp3
Saved: combined_chunk_0008.mp3
Saved: combined_chunk_0009.mp3
Saved: combined_chunk_0010.mp3
Saved: combined_chunk_0011.mp3
Saved: combined_chunk_0012.mp3
Saved: combined_chunk_0013.mp3
Saved: combined_chunk_0014.mp3
Saved: combined_chunk_0015.mp3
Saved: combined_chunk_0016.mp3
Saved: combined_chunk_0017.mp3
Saved: combined_chunk_0018.mp3
Saved: combined_chunk_0019.mp3
Saved: combined_chunk_0020.mp3
Saved: combined_chunk_0021.mp3
Saved: combined_chunk_0022.mp3
Saved: combined_chunk_0023.mp3
Saved: combined_chunk_0024.mp3
Saved: combined_chunk_0025.mp3
Saved: combined_chunk_0026.mp3
Saved: combined_chunk_0027.mp3
Saved: combined_chunk_0028.mp3
Saved: combined_chunk_0029.mp3
Saved: combined_chunk_0030.mp3
Saved: combined_chunk_0031.mp3
Saved: c

KeyboardInterrupt: 

In [35]:

import os
from googletrans import Translator
import jieba  # Import jieba for Chinese word segmentation
import time
# Directory where your text files are stored
output_dir = "/content/split_audio"  # Update with the correct path

# Initialize Google Translate API (googletrans library)
translator = Translator()
sleep_duration = 0.2
# Function to translate a word using Google Translate API
def translate_word(word):
    time.sleep(sleep_duration)
    try:
        translated = translator.translate(word, src='zh-cn', dest='en')
        if translated.text:
            return translated.text
        else:
            print(f"Warning: No translation for {word}")
            return f"No translation for {word}"
    except Exception as e:
        print(f"Error translating {word}: {e}")
        return f"Translation error for {word}"



# Function to get Pinyin of a Chinese word using pypinyin
def get_pinyin(word):
    from pypinyin import lazy_pinyin  # Import here to avoid unnecessary import if not used
    pinyin = lazy_pinyin(word)  # Convert the word to Pinyin
    return ' '.join(pinyin)  # Join the list into a string

# Function to process all .txt files and generate word meanings
def process_text_files():
    for filename in os.listdir(output_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                sentence = f.read().strip()  # Read sentence from the file
                sentence_translation =  translate_word(sentence)
                # Use jieba to segment the sentence into words
                words = jieba.lcut(sentence)  # lcut() returns a list of words

                # For each word in the sentence, translate or look up its meaning
                word_meanings = {}
                for word in words:
                    meaning = translate_word(word)  # Get the English translation

                    pinyin = get_pinyin(word)  # Get the Pinyin for the word
                    if meaning:
                        word_meanings[word] = {'meaning': meaning, 'pinyin': pinyin}

                # Prepare the formatted output
                formatted_output = f"Sentence: {sentence}\n"
                formatted_output += f"Translation: {sentence_translation}\n\n"
                for word, info in word_meanings.items():
                    formatted_output += f"{word}: [{info['meaning']}] ({info['pinyin']})\n"

                print(formatted_output)

                # Save the formatted output back to the same file
                # with open(file_path, 'w', encoding='utf-8') as txt_file:
                #     txt_file.write(formatted_output)

                # Print confirmation for the saved file
                print(f"Formatted output saved in: {filename}")

# Call the function to process the text files
process_text_files()


Sentence: 这机器就理论上可以进行非常复杂非常大量的数学运算
Translation: This machine can theoretically perform very complex and very large amounts of mathematical operations

这: [this] (zhe)
机器: [machine] (ji qi)
就: [At once] (jiu)
理论: [theory] (li lun)
上: [superior] (shang)
可以: [Can] (ke yi)
进行: [conduct] (jin xing)
非常复杂: [Very complex] (fei chang fu za)
非常: [Very] (fei chang)
大量: [A large number] (da liang)
的: [of] (de)
数学: [math] (shu xue)
运算: [Operation] (yun suan)

Formatted output saved in: combined_chunk_0025.txt
Sentence: 可能算是历史上最值钱的公司通用汽车它市值虽然不是最高
Translation: It may be considered the most valuable company in history. Its market value is not the highest

可能: [possible] (ke neng)
算是: [It's considered] (suan shi)
历史: [history] (li shi)
上: [superior] (shang)
最: [most] (zui)
值钱: [valuable] (zhi qian)
的: [of] (de)
公司: [company] (gong si)
通用汽车: [General Motors] (tong yong qi che)
它: [it] (ta)
市值: [Market value] (shi zhi)
虽然: [Although] (sui ran)
不是: [no] (bu shi)
最高: [Highest] (zui gao)

Formatted output s

KeyboardInterrupt: 

### Zip and download generated clips

In [14]:
import shutil
from google.colab import files

# Specify the folder path
folder_path = "/content/split_audio"

# Create a zip file of the folder
shutil.make_archive('/content/split_audio', 'zip', folder_path)

# Provide a download link for the zip file
files.download('/content/split_audio.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>